In [1]:
#!pip install -r requirements.txt

In [5]:
# --------------------------
# SIMPLE MODEL EVALUATION (MINIMAL CHANGES)
# --------------------------
def simple_model_evaluation():
    """Simple evaluation with minimal changes from original code."""
    
    # Generate synthetic data for testing
    np.random.seed(42)
    n_samples = 144
    horizon = 12
    
    t = np.arange(n_samples)
    trend = 100 + 0.5 * t
    seasonal = 20 * np.sin(2 * np.pi * t / 12)
    noise = np.random.normal(0, 5, n_samples)
    data = trend + seasonal + noise
    
    df = pd.DataFrame({'passengers': data})
    target_col = 'passengers'
    
    train_df = df.iloc[:-horizon]
    val_df = df.iloc[-horizon:]
    
    # Models to test (updated class names)
    models_to_test = [
        compose.Pipeline(
            ('lag_features', RobustLagFeatures(lags=3)),
            ('scale', preprocessing.StandardScaler()),
            ('model', linear_model.LinearRegression(optimizer=optim.SGD(lr=0.001), l2=0.01))
        ),
        compose.Pipeline(
            ('lag_features', RobustLagFeatures(lags=3)),
            ('scale', preprocessing.StandardScaler()),
            ('model', linear_model.BayesianLinearRegression(alpha=1, beta=1))
        ),
        linear_model.PARegressor(C=0.01, mode=2),
        neighbors.KNNRegressor(n_neighbors=5)
    ]
    
    # --------------------------
    # EVALUATION
    # --------------------------
    results = {}
    for model in models_to_test:
        model_name = type(model).__name__ if not isinstance(model, compose.Pipeline) else type(model['model']).__name__
        lag_transformer = RobustLagFeatures(lags=3)
        
        # initialize metrics
        mae = metrics.MAE()
        mape = metrics.MAPE()
        rmse = metrics.RMSE()
        r2 = metrics.R2()
        
        # TRAIN & ONLINE EVALUATION
        for _, row in train_df.iterrows():
            x = {}
            y = row[target_col]
            lag_transformer.learn_one(x, y)
            features = lag_transformer.transform_one(x)
            
            # predict before learning (online)
            y_pred = model.predict_one(features)
            if y_pred is not None:
                mae.update(y, y_pred)
                mape.update(y, y_pred)
                rmse.update(y, y_pred)
                r2.update(y, y_pred)
            
            # learn from current sample
            model.learn_one(features, y)
        
        # VALIDATE
        for _, row in val_df.iterrows():
            x = {}
            y = row[target_col]
            lag_transformer.learn_one(x, y)
            features = lag_transformer.transform_one(x)
            y_pred = model.predict_one(features)
            if y_pred is not None:
                mae.update(y, y_pred)
                mape.update(y, y_pred)
                rmse.update(y, y_pred)
                r2.update(y, y_pred)
            model.learn_one(features, y)
        
        results[model_name] = {
            "MAE": mae.get(),
            "MAPE": mape.get(),
            "RMSE": rmse.get(),
            "R2": r2.get()
        }
    
    # --------------------------
    # PRINT RESULTS
    # --------------------------
    print("\n=== Benchmark Results ===")
    for name, metrics_dict in results.items():
        print(f"{name}: ", end="")
        print(", ".join([f"{k}={v:.2f}" for k, v in metrics_dict.items()]))
    
    # --------------------------
    # ONLINE LEARNING SIMULATION EXAMPLE
    # --------------------------
    print("\n=== Online Learning Simulation with KNN ===")
    lag_transformer = RobustLagFeatures(lags=3)
    online_model = compose.Pipeline(
        ('lag_features', lag_transformer),
        ('scale', preprocessing.StandardScaler()),  # optional for KNN
        ('model', neighbors.KNNRegressor(n_neighbors=5))
    )
    
    # metrics for online simulation
    online_mae = metrics.MAE()
    online_mape = metrics.MAPE()
    online_rmse = metrics.RMSE()
    online_r2 = metrics.R2()
    
    for i, row in df.iterrows():
        x = {}
        y = row[target_col]
        lag_transformer.learn_one(x, y)
        features = lag_transformer.transform_one(x)
        y_pred = online_model.predict_one(features)
        if y_pred is not None:
            online_mae.update(y, y_pred)
            online_mape.update(y, y_pred)
            online_rmse.update(y, y_pred)
            online_r2.update(y, y_pred)
        online_model.learn_one(features, y)
        
        if i < 5:
            print(f"Step {i}, True={y:.2f}, Predicted={y_pred:.2f if y_pred else None}")
    
    print(f"Online learning MAE with KNN after full pass: {online_mae.get():.2f}")
    print(f"Online learning MAPE with KNN: {online_mape.get():.2f}")
    print(f"Online learning RMSE with KNN: {online_rmse.get():.2f}")
    print(f"Online learning R2 with KNN: {online_r2.get():.2f}")

# --------------------------
# RUN COMPLETE EVALUATION
# --------------------------
if __name__ == "__main__":
    try:
        # Run robust simulation first
        print("=== PART 1: Robust Simulation ===")
        final_model, metrics_obj, errors = robust_online_learning_simulation()
        
        # Run original style evaluation with CSV
        print("\n=== PART 2: Original Style Evaluation ===")
        csv_results = run_original_style_evaluation()
        
        print("\nSystem completed all evaluations successfully!")
        
    except Exception as e:
        logger.error(f"Evaluation failed: {e}")
        print("System needs additional improvements")# --------------------------
# ROBUST ONLINE LEARNING EXAMPLE
# --------------------------
import pandas as pd
import numpy as np
import logging
from collections import deque
from river import linear_model, preprocessing, compose, metrics, optim, base
from typing import Dict, Any, Optional

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class RobustLagFeatures(base.Transformer):
    """Robust lag features with comprehensive error handling."""
    
    def __init__(self, lags=3, max_history=1000, buffer=None):
        super().__init__()  # Initialize base transformer
        self.lags = max(1, int(lags))  # ensure positive integer
        self.max_history = max(10, int(max_history))  # minimum sensible buffer
        self.buffer = buffer if buffer is not None else deque(maxlen=self.max_history)
        self.error_count = 0
        
    def learn_one(self, x, y=None):
        try:
            # Validate and clean input
            if y is not None:
                if isinstance(y, (int, float)) and not (np.isnan(y) or np.isinf(y)):
                    self.buffer.append(float(y))
                else:
                    logger.warning(f"Invalid target value: {y}, skipping")
                    self.error_count += 1
            return self
        except Exception as e:
            logger.error(f"Error in learn_one: {e}")
            self.error_count += 1
            return self
    
    def transform_one(self, x):
        try:
            features = {}
            history_list = list(self.buffer)
            
            # Create lag features safely
            for i in range(1, self.lags + 1):
                if len(history_list) >= i:
                    val = history_list[-(i)]
                    features[f'lag_{i}'] = float(val) if val is not None else 0.0
                else:
                    features[f'lag_{i}'] = 0.0
                    
            return features
        except Exception as e:
            logger.error(f"Error in transform_one: {e}")
            # Return safe default features
            return {f'lag_{i}': 0.0 for i in range(1, self.lags + 1)}

class SafeMetrics:
    """Wrapper for safe metric calculations."""
    
    def __init__(self):
        self.mae = metrics.MAE()
        self.rmse = metrics.RMSE()
        self.r2 = metrics.R2()
        self.count = 0
        
    def update(self, y_true, y_pred):
        try:
            # Validate inputs
            if not all(isinstance(val, (int, float)) for val in [y_true, y_pred]):
                return
            if any(np.isnan(val) or np.isinf(val) for val in [y_true, y_pred]):
                return
                
            self.mae.update(y_true, y_pred)
            self.rmse.update(y_true, y_pred)
            self.r2.update(y_true, y_pred)
            self.count += 1
            
        except Exception as e:
            logger.warning(f"Metric update failed: {e}")
    
    def get_results(self) -> Dict[str, float]:
        try:
            return {
                "MAE": self.mae.get() if self.count > 0 else float('inf'),
                "RMSE": self.rmse.get() if self.count > 0 else float('inf'),
                "R2": self.r2.get() if self.count > 0 else -float('inf'),
                "Count": self.count
            }
        except:
            return {"MAE": float('inf'), "RMSE": float('inf'), "R2": -float('inf'), "Count": 0}

def create_robust_model() -> compose.Pipeline:
    """Create a robust model pipeline with error handling."""
    return compose.Pipeline(
        ('lag_features', RobustLagFeatures(lags=3)),
        ('scale', preprocessing.StandardScaler()),
        ('model', linear_model.LinearRegression(
            optimizer=optim.SGD(lr=0.001), 
            l2=0.01
        ))
    )

def safe_predict(model, features) -> Optional[float]:
    """Safely make predictions with fallback."""
    try:
        pred = model.predict_one(features)
        if pred is not None and isinstance(pred, (int, float)):
            if not (np.isnan(pred) or np.isinf(pred)):
                return float(pred)
        return None
    except Exception as e:
        logger.warning(f"Prediction failed: {e}")
        return None

def robust_online_learning_simulation():
    """Demonstrate robust online learning that handles real-world issues."""
    
    logger.info("Starting robust online learning simulation...")
    
    # Generate synthetic data with some problematic cases
    np.random.seed(42)
    n_samples = 200
    
    # Base trend with seasonality
    t = np.arange(n_samples)
    trend = 100 + 0.5 * t
    seasonal = 20 * np.sin(2 * np.pi * t / 12)
    noise = np.random.normal(0, 5, n_samples)
    
    data = trend + seasonal + noise
    
    # Inject some problematic values to test robustness
    problematic_indices = [25, 50, 75, 125, 175]
    original_values = data[problematic_indices].copy()
    
    data[25] = np.nan        # Missing value
    data[50] = np.inf        # Infinite value
    data[75] = -999999       # Extreme outlier
    data[125] = None         # None value
    # data[175] stays normal for comparison
    
    # Initialize robust components
    model = create_robust_model()
    safe_metrics = SafeMetrics()
    lag_transformer = RobustLagFeatures(lags=3)
    
    predictions = []
    actual_values = []
    error_log = []
    
    print("\n=== Robust Online Learning with Error Injection ===")
    print("Injected problematic values at indices: 25(NaN), 50(Inf), 75(outlier), 125(None)")
    print("\nFirst 10 steps + problematic cases:")
    
    for i, y in enumerate(data):
        try:
            # Prepare features
            x = {}  # empty features dict as in original
            
            # Update lag transformer (with error handling)
            lag_transformer.learn_one(x, y)
            features = lag_transformer.transform_one(x)
            
            # Make prediction
            y_pred = safe_predict(model, features)
            
            # Update metrics if prediction is valid
            if y_pred is not None and isinstance(y, (int, float)) and not (np.isnan(y) or np.isinf(y)):
                safe_metrics.update(y, y_pred)
                predictions.append(y_pred)
                actual_values.append(y)
            else:
                if i in problematic_indices:
                    error_msg = f"Step {i}: Handled problematic value y={y}"
                    error_log.append(error_msg)
                    if i < 10 or i in problematic_indices:
                        print(error_msg)
                predictions.append(None)
                actual_values.append(y if isinstance(y, (int, float)) and not (np.isnan(y) or np.isinf(y)) else None)
            
            # Learn from current sample (model handles its own errors)
            try:
                if isinstance(y, (int, float)) and not (np.isnan(y) or np.isinf(y)):
                    model.learn_one(features, y)
            except Exception as e:
                logger.warning(f"Model learning failed at step {i}: {e}")
            
            # Print progress for first 10 steps and problematic cases
            if i < 10 or i in problematic_indices:
                status = "✓" if y_pred is not None else "✗"
                pred_str = f"{y_pred:8.2f}" if y_pred is not None else "    None"
                print(f"Step {i:3d}: True={y:8.2f}, Pred={pred_str}, Status={status}")
                
        except Exception as e:
            error_msg = f"Step {i}: Critical error - {e}"
            error_log.append(error_msg)
            logger.error(error_msg)
            # Continue execution despite errors
            predictions.append(None)
            actual_values.append(None)
    
    # Final results
    final_metrics = safe_metrics.get_results()
    valid_predictions = sum(1 for p in predictions if p is not None)
    
    print(f"\n=== Simulation Results ===")
    print(f"Total samples processed: {len(data)}")
    print(f"Valid predictions made: {valid_predictions}")
    print(f"Error rate: {len(error_log)/len(data)*100:.1f}%")
    print(f"Lag transformer errors: {lag_transformer.error_count}")
    
    for metric, value in final_metrics.items():
        if metric == "Count":
            print(f"{metric}: {value}")
        else:
            print(f"{metric}: {value:.3f}")
    
    print(f"\n=== Error Recovery Examples ===")
    for error in error_log[:3]:  # Show first 3 errors
        print(f"- {error}")
    
    print(f"\n✅ Simulation completed successfully despite {len(error_log)} problematic values!")
    print("🔄 Model continued learning and adapting throughout all errors")
    
    return model, safe_metrics, error_log

# --------------------------
# MODELS TO TEST (ROBUST VERSIONS)
# --------------------------
from river import neighbors

def create_models_to_test():
    """Create robust model list for benchmarking."""
    return [
        compose.Pipeline(
            ('lag_features', RobustLagFeatures(lags=3)),
            ('scale', preprocessing.StandardScaler()),
            ('model', linear_model.LinearRegression(optimizer=optim.SGD(lr=0.001), l2=0.01))
        ),
        compose.Pipeline(
            ('lag_features', RobustLagFeatures(lags=3)),
            ('scale', preprocessing.StandardScaler()),
            ('model', linear_model.BayesianLinearRegression(alpha=1, beta=1))
        ),
        linear_model.PARegressor(C=0.01, mode=2),
        neighbors.KNNRegressor(n_neighbors=5)
    ]

# --------------------------
# ROBUST EVALUATION FUNCTION
# --------------------------
def robust_model_evaluation():
    """Evaluate multiple models with robust error handling."""
    
    # Generate synthetic data for evaluation
    np.random.seed(42)
    n_samples = 144  # 12 years of monthly data
    horizon = 12
    
    # Create synthetic time series
    t = np.arange(n_samples)
    trend = 100 + 0.5 * t
    seasonal = 20 * np.sin(2 * np.pi * t / 12)
    noise = np.random.normal(0, 5, n_samples)
    data = trend + seasonal + noise
    
    # Create DataFrame
    df = pd.DataFrame({'passengers': data})
    target_col = 'passengers'
    
    train_df = df.iloc[:-horizon]
    val_df = df.iloc[-horizon:]
    
    models_to_test = create_models_to_test()
    results = {}
    
    print("\n=== Robust Model Benchmark ===")
    
    for model in models_to_test:
        model_name = type(model).__name__ if not isinstance(model, compose.Pipeline) else type(model['model']).__name__
        lag_transformer = RobustLagFeatures(lags=3)
        
        # Initialize safe metrics
        safe_metrics_obj = SafeMetrics()
        mape = metrics.MAPE()  # Keep MAPE separate for compatibility
        
        print(f"Training {model_name}...")
        
        # TRAIN & ONLINE EVALUATION
        for _, row in train_df.iterrows():
            try:
                x = {}
                y = row[target_col]
                lag_transformer.learn_one(x, y)
                features = lag_transformer.transform_one(x)
                
                # Predict before learning (online)
                y_pred = safe_predict(model, features)
                if y_pred is not None:
                    safe_metrics_obj.update(y, y_pred)
                    try:
                        mape.update(y, y_pred)
                    except:
                        pass  # Handle MAPE division by zero
                
                # Learn from current sample
                if isinstance(y, (int, float)) and not (np.isnan(y) or np.isinf(y)):
                    model.learn_one(features, y)
                    
            except Exception as e:
                logger.warning(f"Training error for {model_name}: {e}")
                continue
        
        # VALIDATE
        for _, row in val_df.iterrows():
            try:
                x = {}
                y = row[target_col]
                lag_transformer.learn_one(x, y)
                features = lag_transformer.transform_one(x)
                y_pred = safe_predict(model, features)
                if y_pred is not None:
                    safe_metrics_obj.update(y, y_pred)
                    try:
                        mape.update(y, y_pred)
                    except:
                        pass
                
                if isinstance(y, (int, float)) and not (np.isnan(y) or np.isinf(y)):
                    model.learn_one(features, y)
                    
            except Exception as e:
                logger.warning(f"Validation error for {model_name}: {e}")
                continue
        
        # Collect results
        model_results = safe_metrics_obj.get_results()
        try:
            model_results["MAPE"] = mape.get()
        except:
            model_results["MAPE"] = float('inf')
            
        results[model_name] = model_results
    
    return results

# --------------------------
# PRINT BENCHMARK RESULTS
# --------------------------
def print_benchmark_results(results):
    """Print benchmark results in a clean format."""
    print("\n=== Benchmark Results ===")
    for name, metrics_dict in results.items():
        print(f"{name}: ", end="")
        metrics_str = []
        for k, v in metrics_dict.items():
            if k != "Count":
                if v == float('inf') or v == -float('inf'):
                    metrics_str.append(f"{k}=inf")
                else:
                    metrics_str.append(f"{k}={v:.2f}")
        print(", ".join(metrics_str))

# --------------------------
# RUN COMPLETE EVALUATION
# --------------------------
if __name__ == "__main__":
    try:
        # Run robust simulation first
        print("=== PART 1: Robust Simulation ===")
        final_model, metrics_obj, errors = robust_online_learning_simulation()
        
        # Run model benchmark
        print("\n=== PART 2: Model Benchmark ===")
        benchmark_results = robust_model_evaluation()
        print_benchmark_results(benchmark_results)
        
        print(f"\nSystem completed both robustness test and model benchmarking successfully!")
        
    except Exception as e:
        logger.error(f"Evaluation failed: {e}")
        print("System needs additional improvements")

2025-08-31 01:59:49,388 - INFO - Starting robust online learning simulation...
2025-08-31 01:59:49,389 - WARNING - Invalid target value: nan, skipping
2025-08-31 01:59:49,390 - WARNING - Invalid target value: inf, skipping
2025-08-31 01:59:49,392 - WARNING - Invalid target value: nan, skipping
2025-08-31 01:59:49,394 - ERROR - Evaluation failed: name 'run_original_style_evaluation' is not defined
2025-08-31 01:59:49,395 - INFO - Starting robust online learning simulation...
2025-08-31 01:59:49,396 - WARNING - Invalid target value: nan, skipping
2025-08-31 01:59:49,397 - WARNING - Invalid target value: inf, skipping
2025-08-31 01:59:49,399 - WARNING - Invalid target value: nan, skipping


=== PART 1: Robust Simulation ===

=== Robust Online Learning with Error Injection ===
Injected problematic values at indices: 25(NaN), 50(Inf), 75(outlier), 125(None)

First 10 steps + problematic cases:
Step   0: True=  102.48, Pred=    0.00, Status=✓
Step   1: True=  109.81, Pred=    2.05, Status=✓
Step   2: True=  121.56, Pred=    4.20, Status=✓
Step   3: True=  129.12, Pred=    6.55, Status=✓
Step   4: True=  118.15, Pred=    9.00, Status=✓
Step   5: True=  111.33, Pred=   11.19, Status=✓
Step   6: True=  110.90, Pred=   13.19, Status=✓
Step   7: True=   97.34, Pred=   15.14, Status=✓
Step   8: True=   84.33, Pred=   16.79, Status=✓
Step   9: True=   87.21, Pred=   18.14, Status=✓
Step 25: Handled problematic value y=nan
Step  25: True=     nan, Pred=   41.52, Status=✓
Step 50: Handled problematic value y=inf
Step  50: True=     inf, Pred=   70.58, Status=✓
Step  75: True=-999999.00, Pred=   94.19, Status=✓
Step 125: Handled problematic value y=nan
Step 125: True=     nan, Pred=-7